In [50]:
import pandas as pd
import regex
import os
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
import gensim

In [51]:
import re, sys

def is_time_stamp(l):
    if l[:2].isnumeric() and l[2] == ':':
        return True
    return False

def has_letters(line):
    if re.search('[a-zA-Z]', line):
        return True
    return False

def has_no_text(line):
    l = line.strip()
    if not len(l):
        return True
    if l.isnumeric():
        return True
    if is_time_stamp(l):
        return True
    if l[0] == '(' and l[-1] == ')':
        return True
    if not has_letters(line):
        return True
    return False

def is_lowercase_letter_or_comma(letter):
    if letter.isalpha() and letter.lower() == letter:
        return True
    if letter == ',':
        return True
    return False

def clean_up(lines):
    new_lines = []
    for line in lines[1:]:
        line = line.replace("\n","")
        if has_no_text(line):
            continue
        elif len(new_lines) and is_lowercase_letter_or_comma(line[0]):
            #combine with previous line
            new_lines[-1] = new_lines[-1].strip() + ' ' + line
        else:
            #append line
            new_lines.append(line)
    return new_lines

def read_srt(file_name):
    with open(file_name, encoding="utf-8", errors='replace') as f:
        lines = f.readlines()
        new_lines = clean_up(lines)
        return new_lines



In [52]:
path = "orignal dump"
filenames = os.listdir(path)
df = pd.DataFrame({"File": filenames,
                   "Season": [f.split(".")[1][:3] for f in filenames],
                   "SeasonID": [int(f.split(".")[1][1:3]) for f in filenames],
                   "Episode": [f.split(".")[1][-3:] for f in filenames],
                   "EpisodeID": [int(f.split(".")[1][-2:]) for f in filenames],
                   "DialogueLines": [read_srt(os.path.join(path,f)) for f in filenames]})
df.head()

,File,Season,SeasonID,Episode,EpisodeID,DialogueLines
0,Suits.S01E01.srt,S01,1,E01,1,"[Gerald Tate's here., He wants to know what's ..."
1,Suits.S01E02.srt,S01,1,E02,2,"[What is that, three in a row?, That would be ..."
2,Suits.S01E03.srt,S01,1,E03,3,"[♪ It's a perfect day ♪, ♪ To go and tie one o..."
3,Suits.S01E04.srt,S01,1,E04,4,"[Harvard trivia, the lightning round., [Snaps]..."
4,Suits.S01E05.srt,S01,1,E05,5,"[Ross., Triple double, courtesy of, America's ..."


In [53]:
df.groupby("SeasonID").count()

,File,Season,Episode,EpisodeID,DialogueLines
SeasonID,,,,,
1,12,12,12,12,12
2,16,16,16,16,16
3,16,16,16,16,16
4,16,16,16,16,16
5,16,16,16,16,16
6,16,16,16,16,16
7,16,16,16,16,16
8,16,16,16,16,16
9,6,6,6,6,6


In [54]:
df["Text"] = df["DialogueLines"].apply(lambda x: " ".join(list(x)))
df['Word_count'] = df["Text"].apply(lambda x: len(str(x).split(" ")))
df['Char_count'] = df["Text"].str.len() 
df['Stopwords'] = df["Text"].apply(lambda x: len([x for x in x.split() if x in stop]))

In [55]:
df.head()

,File,Season,SeasonID,Episode,EpisodeID,DialogueLines,Text,Word_count,Char_count,Stopwords
0,Suits.S01E01.srt,S01,1,E01,1,"[Gerald Tate's here., He wants to know what's ...",Gerald Tate's here. He wants to know what's ha...,9371,48607,3501
1,Suits.S01E02.srt,S01,1,E02,2,"[What is that, three in a row?, That would be ...","What is that, three in a row? That would be fo...",6026,30980,2222
2,Suits.S01E03.srt,S01,1,E03,3,"[♪ It's a perfect day ♪, ♪ To go and tie one o...",♪ It's a perfect day ♪ ♪ To go and tie one on ...,5843,30808,2150
3,Suits.S01E04.srt,S01,1,E04,4,"[Harvard trivia, the lightning round., [Snaps]...","Harvard trivia, the lightning round. [Snaps] G...",6469,34831,2307
4,Suits.S01E05.srt,S01,1,E05,5,"[Ross., Triple double, courtesy of, America's ...","Ross. Triple double, courtesy of America's fav...",5961,31022,2069


In [58]:
df.to_csv("Data.csv", index_col=False)

TypeError: to_csv() got an unexpected keyword argument 'index_col'

In [64]:
df[df["Word_count"]<5000]

,File,Season,SeasonID,Episode,EpisodeID,DialogueLines,Text,Word_count,Char_count,Stopwords
